# Predicting Home Sale Price

This will attempt to be similar to the Zestimate used on Zillow, the Redfin estimate used on Redfin, or any of the valuations provided by Collateral Analytics, CoreLogic, or Quantarium on Realtor.com.

I will create a regression model and use it to predict valuation and compare to the other valuations on currently listed homes.

In [155]:
%run src/imports.py

In [156]:
sns.set(rc={'figure.figsize':(16,4)})

In [157]:
neighborhoods_cleaned = pd.read_pickle('data/neighborhoods_cleaned.pkl')
neighborhoods_cleaned.reset_index(inplace=True)
neighborhoods_cleaned.drop(['index','DAYS ON MARKET','PROPERTY TYPE','ADDRESS','CITY','STATE OR PROVINCE'],axis=1,inplace=True)
#neighborhoods_cleaned = neighborhoods_cleaned.merge(pd.get_dummies(neighborhoods_cleaned['LOCATION'],drop_first=True), how='outer',left_index=True,right_index=True)
neighborhoods_cleaned.drop(['LOCATION'],axis=1,inplace=True)
neighborhoods_cleaned['SOLD DATE'] = pd.to_datetime(neighborhoods_cleaned['SOLD DATE'])
neighborhoods_cleaned['MONTH'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).month
neighborhoods_cleaned['YEAR'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).year
neighborhoods_cleaned.drop(['SOLD DATE'],axis=1,inplace=True)

In [158]:
neighborhoods_cleaned

,ZIP OR POSTAL CODE,PRICE,SQUARE FEET,LOT SIZE,YEAR BUILT,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS PER SQFT * 1000,MONTH,YEAR
0,78257,345000,1989.0,5662.0,2005.0,230.0,29.647801,-98.614186,2.011061,9,2020
1,78256,337500,2166.0,9583.0,2004.0,54.0,29.650304,-98.629082,2.770083,6,2020
2,78256,375500,2969.0,6605.0,2004.0,54.0,29.651842,-98.630650,2.189289,3,2021
3,78255,462000,2716.0,9147.0,2014.0,95.0,29.617650,-98.643430,2.577320,10,2018
4,78256,624000,3012.0,75794.0,2000.0,25.0,29.620646,-98.622295,1.826029,2,2021
...,...,...,...,...,...,...,...,...,...,...,...
20007,78222,153000,1577.0,6969.0,1983.0,NaN,29.383410,-98.376520,3.170577,9,2019
20008,78222,189900,1703.0,4791.0,2018.0,25.0,29.386700,-98.379900,3.523194,7,2018
20009,78222,128000,1102.0,5662.0,1985.0,NaN,29.383092,-98.376562,4.537205,3,2019
20010,78222,196499,2324.0,5662.0,2014.0,6.0,29.379346,-98.370269,3.012048,3,2019


In [159]:
neighborhoods_cleaned.dropna(inplace=True)

In [160]:
y = neighborhoods_cleaned['PRICE'].values
X = neighborhoods_cleaned.drop('PRICE',axis=1).values

In [161]:
neighborhoods_cleaned

,ZIP OR POSTAL CODE,PRICE,SQUARE FEET,LOT SIZE,YEAR BUILT,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS PER SQFT * 1000,MONTH,YEAR
0,78257,345000,1989.0,5662.0,2005.0,230.0,29.647801,-98.614186,2.011061,9,2020
1,78256,337500,2166.0,9583.0,2004.0,54.0,29.650304,-98.629082,2.770083,6,2020
2,78256,375500,2969.0,6605.0,2004.0,54.0,29.651842,-98.630650,2.189289,3,2021
3,78255,462000,2716.0,9147.0,2014.0,95.0,29.617650,-98.643430,2.577320,10,2018
4,78256,624000,3012.0,75794.0,2000.0,25.0,29.620646,-98.622295,1.826029,2,2021
...,...,...,...,...,...,...,...,...,...,...,...
20002,78222,137000,2773.0,5227.0,2005.0,6.0,29.379318,-98.373976,2.163722,6,2018
20003,78222,165000,2085.0,8276.0,2014.0,7.0,29.388351,-98.370269,3.117506,1,2020
20006,78222,165900,1246.0,5662.0,2018.0,7.0,29.374355,-98.385532,4.815409,2,2019
20008,78222,189900,1703.0,4791.0,2018.0,25.0,29.386700,-98.379900,3.523194,7,2018


-----
Data Density  
= 11,654 / 11
= 1000
-----

## Building a Model

In [211]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [213]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
y_predict = rfr.predict(X_test)
print("score:", rfr.score(X_test, y_test))
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

score: 0.9104055679882893
MSE: 1.048657e+09


In [214]:
y_predict

array([199814.78, 182766.94, 228420.09, ..., 261187.97, 212646.42,
       225981.83])

In [215]:
y_test

array([193000, 186300, 217457, ..., 250000, 212900, 259220])

In [172]:
search_space = [{'regressor__alpha': 10 ** (np.linspace(-5, 5, 10)), 
                 'regressor__l1_ratio': np.linspace(0,1,6)}]

# l_lasso / (l_lasso + l_ridge)
# if l_lasso = 0: 0 / 1 = 0 (this is ridge regression)
# if l_ridge = 0: l_lasso / l_lasso = 1 (this is LASSSO regression)

pipe = Pipeline([('scaler', StandardScaler()), ('regressor', ElasticNet())])

enet_cv = GridSearchCV(pipe, 
                      search_space, 
                      cv=KFold(10, shuffle=True), 
                      scoring='neg_mean_squared_error',
                      return_train_score = True)

In [173]:
enet_cv.fit(X_train,y_train)

enet_cv.best_params_

/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14569338064369.205, tolerance: 10111310072.75697
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14530357101143.602, tolerance: 10000945617.697178
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14300817914794.814, tolerance: 9926970132.732944
  model = cd_fast.enet_coordinate_descent(
/Users/randallbloomquist/opt/anac

{'regressor__alpha': 46.41588833612782, 'regressor__l1_ratio': 1.0}

In [174]:
y_predict = enet_cv.predict(X_test)
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

MSE: 4.069101e+13


In [175]:
ridge = Ridge(alpha=1)
ridge.fit(X_train,y_train)
y_predict = ridge.predict(X_test)
print("score:", ridge.score(X_test, y_test))
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

score: -3319.9261514277987
MSE: 4.081817e+13


In [176]:
X_train.shape

(8740, 10)

In [177]:
y_train.shape

(8740,)

In [179]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_predict = lr.predict(X_test)
print("score:", lr.score(X_test, y_test))
print('MSE: {:2e}'.format(mean_squared_error(y_test, y_predict)))

score: -3313.4886477498917
MSE: 4.073905e+13


In [103]:
np.arange(-5,5,1)

array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4])

In [110]:
type(y_train)

numpy.ndarray

In [180]:
kf = KFold()
skf = StratifiedKFold()
ridge_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())])
ridge_grid_params = {'regressor__alpha': np.arange(0,50,1)}
ridge_grid = GridSearchCV(ridge_pipe, ridge_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
ridge_grid.fit(X_train, y_train)
print("Ridge Accuracy: {:.2e}".format(ridge_grid.score(X_test, y_test)))

Ridge Accuracy: -4.07e+13


In [181]:
kf = KFold()
skf = StratifiedKFold()
lasso_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Lasso())])
lasso_grid_params = {'regressor__alpha': np.arange(0.1,0.9,0.1)}
lasso_grid = GridSearchCV(lasso_pipe, lasso_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
lasso_grid.fit(X_train, y_train)
print("Lasso Accuracy: {:.2e}".format(lasso_grid.score(X_test, y_test)))

Lasso Accuracy: -4.07e+13


In [217]:
kf = KFold()
skf = StratifiedKFold()
gbr_pipe = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(loss='huber'))])
gbr_grid_params = {'regressor__alpha': np.arange(0.1,0.9,0.3),
                  'regressor__max_depth': np.arange(1,9,3),
                  'regressor__learning_rate': np.arange(0.01,0.2,0.05)}
gbr_grid = GridSearchCV(gbr_pipe, gbr_grid_params, cv=kf, scoring = 'neg_mean_squared_error')
gbr_grid.fit(X_train, y_train)
print("gbr Accuracy: {:.2e}".format(gbr_grid.score(X_test, y_test)))

gbr Accuracy: -8.31e+08


In [182]:
kf = KFold()
skf = StratifiedKFold()
gbr_pipe2 = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(loss='huber'))])
gbr_grid_params2 = {'regressor__alpha': np.arange(0.1,0.9,0.3)}
gbr_grid2 = GridSearchCV(gbr_pipe2, gbr_grid_params2, cv=kf, scoring = 'neg_mean_squared_error')
gbr_grid2.fit(X_train, y_train)
print("gbr Accuracy: {:.2e}".format(gbr_grid.score(X_test, y_test)))

gbr Accuracy: -1.52e+09


In [183]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [184]:
np.arange(0,10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [185]:
X_df = pd.DataFrame(X)

In [186]:
X_df.corr() > 0.2

,0,1,2,3,4,5,6,7,8,9
0,True,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,True,False,False
4,False,False,False,False,True,False,False,False,False,False
5,False,True,False,False,False,True,False,False,False,False
6,False,False,False,False,False,False,True,False,False,False
7,False,False,False,True,False,False,False,True,False,False
8,False,False,False,False,False,False,False,False,True,False
9,False,False,False,False,False,False,False,False,False,True


In [199]:
X_train = sm.add_constant(X_train)

In [200]:
X_opt = X_train[:,np.arange(0,10)]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     2286.
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        22:03:31   Log-Likelihood:            -1.0874e+05
No. Observations:                8740   AIC:                         2.175e+05
Df Residuals:                    8730   BIC:                         2.176e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.401e+06   9.26e+05      1.513      0.130   -4.14e+05    3.22e+06
x1           -90.6304     11.428     -7.930      0.000    -113.033     -68.228
x2           105.7373      1.427     74.093      0.000     102.940     108.535
x3             0.0002      0.000      0.920      0.357      -0.000       0.001
x4           871.5939     77.830     11.199      0.000     719.028    1024.160
x5          1055.0176     21.998     47.960      0.000    1011.896    1098.139
x6          8.278e+04   6999.336     11.827      0.000    6.91e+04    9.65e+04
x7         -1.451e+04   4057.741     -3.576      0.000   -2.25e+04   -6556.809
x8          1.668e+04   1847.358      9.030      0.000    1.31e+04    2.03e+04
x9           328.9729    204.860      1.606      0.108     -72.601     730.547
==============================================================================
Omnibus:                     6365.773   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           719308.317
Skew:                           2.697   Prob(JB):                         0.00
Kurtosis:                      47.115   Cond. No.                     5.07e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.07e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [202]:
X_opt = X_train[:,[0,1,2,4,5,6,7,8,9,10]]

In [203]:
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.713
Method:                 Least Squares   F-statistic:                     2409.
Date:                Wed, 24 Mar 2021   Prob (F-statistic):               0.00
Time:                        22:03:59   Log-Likelihood:            -1.0857e+05
No. Observations:                8740   AIC:                         2.172e+05
Df Residuals:                    8730   BIC:                         2.172e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.625e+07   1.77e+06    -14.793      0.000   -2.97e+07   -2.28e+07
x1           -94.1917     11.221     -8.394      0.000    -116.188     -72.196
x2           108.0740      1.407     76.822      0.000     105.316     110.832
x3           831.0016     76.436     10.872      0.000     681.169     980.834
x4          1033.5446     21.627     47.789      0.000     991.150    1075.939
x5          8.316e+04   6871.368     12.103      0.000    6.97e+04    9.66e+04
x6         -1.834e+04   3989.132     -4.597      0.000   -2.62e+04   -1.05e+04
x7          1.941e+04   1819.451     10.669      0.000    1.58e+04     2.3e+04
x8          1133.9101    205.944      5.506      0.000     730.212    1537.608
x9          1.367e+04    753.404     18.142      0.000    1.22e+04    1.51e+04
==============================================================================
Omnibus:                     6564.327   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           786991.236
Skew:                           2.821   Prob(JB):                         0.00
Kurtosis:                      49.144   Cond. No.                     2.16e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [205]:
y_predict = regressor_OLS.predict(X_test)

In [208]:
print('MSE: {:.2e}'.format(mean_squared_error(y_test, y_predict)))

MSE: 4.21e+24
